In [3]:
import openai
import fitz  # PyMuPDF
# import a chain from langchain.chains
from langchain.chains import ConversationChain
# from langchain.chains import SimpleChain
from langchain.prompts import PromptTemplate
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [4]:
# Set up your OpenAI API key
openai.api_key = 'xx-xxxx'

In [5]:
# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    """
    Extracts all text from a PDF file.
    """
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

In [ ]:
pdf_path = 'star_signs.pdf'
context = extract_text_from_pdf(pdf_path)
# print the first 1000 characters of the text
print(context[:1000])

In [13]:
# Fixed-size chunking method
def fixed_size_chunking(text, chunk_size=512):
    """
    Splits text into fixed-size chunks based on a specified chunk size.
    """
    words = text.split()
    chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

In [ ]:
chunks = fixed_size_chunking(context)  # Chunk the extracted text
# print the first chunk
print(chunks[0])

In [17]:
# Basic retrieval method using TF-IDF
def retrieve_relevant_chunks(chunks, query, top_k=3):
    """
    Retrieves the most relevant chunks based on the user's query using TF-IDF.
    """
    vectorizer = TfidfVectorizer().fit_transform(chunks)
    query_vec = vectorizer.transform([query])
    similarity = cosine_similarity(query_vec, vectorizer).flatten()
    relevant_indices = similarity.argsort()[-top_k:][::-1]
    relevant_chunks = [chunks[i] for i in relevant_indices]
    return relevant_chunks

In [18]:
# Sentence-based chunking method
def sentence_based_chunking(text):
    """
    Splits text into sentences using regular expressions.
    """
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return sentences

In [19]:
# Define the response function
def get_openai_response(prompt):
    """
    This function sends a prompt to the OpenAI API and retrieves the response.
    """
    response = openai.Completion.create(
        engine="gpt-4o",  # The model to use
        prompt=prompt,              # The prompt containing the user input
        max_tokens=150,             # The maximum number of tokens to generate
        n=1,                        # Number of responses to generate
        stop=None,                  # Stop sequence for the model
        temperature=0.7,            # Sampling temperature for response generation
    )
    return response.choices[0].text.strip()  # Return the generated response

In [20]:
# Define a prompt template that includes the context
prompt_template = PromptTemplate(template="Context: {context}\n\nThe user says: {input}\nAI assistant replies:", input_variables=["context", "input"])

In [43]:
# Define a conversation chain class
class OpenAIChatbotChain(ConversationChain):
    def __init__(self, prompt_template, chunks):
        self.chunks = chunks
        self.prompt_template = prompt_template

In [44]:
# Create a conversation chain instance
chatbot_chain = OpenAIChatbotChain(prompt_template, chunks)

ValueError: "OpenAIChatbotChain" object has no field "chunks"